In [80]:
from fakephys import model, noise
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
output_notebook()

import numpy as np
from scipy.signal import welch

Loading BokehJS ...

In [81]:
# GLOBAL params
t = 2.5
times = model.create_times(t, 1e-3)

# Examples

## Models

Examples timecourses, and parameter sets, for each of the models available to simulate ephys signals.

### Sin

The obvious inclusion

In [82]:
X = model.sinwaves(t, 1, f=20, a=1, phase=0)
x = X[0, :]

# -
freqs, psd = welch(x, int(1 / 1e-3), scaling='density', nperseg=1000/2)
m = np.logical_and(freqs > 2, freqs < 100)
freqs = freqs[m]
psd = psd[m]

# -
p1 = figure(plot_width=500, plot_height=250)
p1.line(x=times, y=x, color="black")
p1.xaxis.axis_label = "Time (s)"
p1.yaxis.axis_label = "Activity"

p2 = figure(plot_width=300, plot_height=250)
p2.line(x=freqs, y=psd, color="black")
p2.xaxis.axis_label = "Freq (Hz)"
p2.yaxis.axis_label = "Power (AU)"
show(row(p1, p2))

### A wilson cowan system.

A very simple, but very common way, to capture oscillatory dynamics is in the two-population Wilson-Cowan system.  It features a 'excitatory' population (_E_) and an 'inhibitory' population (_I_), which are coupled to each other and to themselves.  My version of this system features 9 parameters:

1. P : bias term for E
2. Q : bias term for I
3. tau_e : the E synaptic time constant (exponential)
4. tau_i : the I synaptic time constant (exponential)
5. c1 : # connections between E => E (i.e., effective synaptic weight)
6. c2 : # connections between I => E
7. c3 : # connections between I => I
8. c4 : # connections between E => I
9. sigma : the standard deviation of [OU](https://en.wikipedia.org/wiki/Ornstein–Uhlenbeck_process) noise added to both populations.

Despite being a mass system (which typically assume many identical neurons, and slow inputs) WC models are quite expressive. 

Here are a few examples:

#### Gamma

In [83]:
X = model.wc(t, 1, P=4, tau_e=5e-3, tau_i=10e-3, sigma=.01)
x = X[0, :]

# -
freqs, psd = welch(x, int(1 / 1e-3), scaling='density', nperseg=1000/2)
m = np.logical_and(freqs > 2, freqs < 100)
freqs = freqs[m]
psd = psd[m]

# -
p1 = figure(plot_width=500, plot_height=250)
p1.line(x=times, y=x, color="black")
p1.xaxis.axis_label = "Time (s)"
p1.yaxis.axis_label = "Activity"

p2 = figure(plot_width=300, plot_height=250)
p2.line(x=freqs, y=psd, color="black")
p2.xaxis.axis_label = "Freq (Hz)"
p2.yaxis.axis_label = "Power (AU)"
show(row(p1, p2))

INFO       No numerical integration method specified for group 'neurongroup', using method 'euler' (took 0.18s, trying other methods took 0.00s). [brian2.stateupdaters.base.method_choice]


Starting simulation at t=0. s for a duration of 2.5 s
2.5 s (100%) simulated in < 1s


#### Beta

In [84]:
X = model.wc(t, 1, P=4, tau_e=5e-3, tau_i=20e-3, sigma=.01)
x = X[0, :]

# -
freqs, psd = welch(x, int(1 / 1e-3), scaling='density', nperseg=1000/2)
m = np.logical_and(freqs > 2, freqs < 100)
freqs = freqs[m]
psd = psd[m]

# -
p1 = figure(plot_width=500, plot_height=250)
p1.line(x=times, y=x, color="black")
p1.xaxis.axis_label = "Time (s)"
p1.yaxis.axis_label = "Activity"

p2 = figure(plot_width=300, plot_height=250)
p2.line(x=freqs, y=psd, color="black")
p2.xaxis.axis_label = "Freq (Hz)"
p2.yaxis.axis_label = "Power (AU)"
show(row(p1, p2))

INFO       No numerical integration method specified for group 'neurongroup_1', using method 'euler' (took 0.16s, trying other methods took 0.00s). [brian2.stateupdaters.base.method_choice]


Starting simulation at t=0. s for a duration of 2.5 s
2.5 s (100%) simulated in < 1s


#### Alpha

In [85]:
X = model.wc(t, 1, P=4, tau_e=10e-3, tau_i=60e-3, sigma=.01)
x = X[0, :]

# -
freqs, psd = welch(x, int(1 / 1e-3), scaling='density', nperseg=1000/2)
m = np.logical_and(freqs > 2, freqs < 100)
freqs = freqs[m]
psd = psd[m]

# -
p1 = figure(plot_width=500, plot_height=250)
p1.line(x=times, y=x, color="black")
p1.xaxis.axis_label = "Time (s)"
p1.yaxis.axis_label = "Activity"

p2 = figure(plot_width=300, plot_height=250)
p2.line(x=freqs, y=psd, color="black")
p2.xaxis.axis_label = "Freq (Hz)"
p2.yaxis.axis_label = "Power (AU)"
show(row(p1, p2))

INFO       No numerical integration method specified for group 'neurongroup', using method 'euler' (took 0.12s, trying other methods took 0.00s). [brian2.stateupdaters.base.method_choice]


Starting simulation at t=0. s for a duration of 2.5 s
2.5 s (100%) simulated in < 1s


#### Unstable oscillations.

All the examples above feature oscillations that are quite stable in time. By inverting the ratio of the time constants on E-I connections, you can produces a dampened oscillation (see first cell below). This type of oscillation has the curious propery of becoming *somewhat* stabilized by introducing a very small amount of noise (see second cell).

In [86]:
# No noise
X = model.wc(t, 5, P=4, tau_e=40e-3, tau_i=35e-3, sigma=.0)
x = X[0, :]

# -
freqs, psd = welch(x, int(1 / 1e-3), scaling='density', nperseg=1000/2)
m = np.logical_and(freqs > 2, freqs < 100)
freqs = freqs[m]
psd = psd[m]

# -
p1 = figure(plot_width=500, plot_height=250)
p1.line(x=times, y=x, color="black")
p1.xaxis.axis_label = "Time (s)"
p1.yaxis.axis_label = "Activity"

p2 = figure(plot_width=300, plot_height=250)
p2.line(x=freqs, y=psd, color="black")
p2.xaxis.axis_label = "Freq (Hz)"
p2.yaxis.axis_label = "Power (AU)"
show(row(p1, p2))

INFO       No numerical integration method specified for group 'neurongroup_1', using method 'euler' (took 0.19s, trying other methods took 0.00s). [brian2.stateupdaters.base.method_choice]


Starting simulation at t=0. s for a duration of 2.5 s
2.5 s (100%) simulated in < 1s


In [87]:
# With noise
X = model.wc(t, 5, P=4, tau_e=40e-3, tau_i=35e-3, sigma=.01)
x = X[0, :]

# -
freqs, psd = welch(x, int(1 / 1e-3), scaling='density', nperseg=1000/2)
m = np.logical_and(freqs > 2, freqs < 100)
freqs = freqs[m]
psd = psd[m]

# -
p1 = figure(plot_width=500, plot_height=250)
p1.line(x=times, y=x, color="black")
p1.xaxis.axis_label = "Time (s)"
p1.yaxis.axis_label = "Activity"

p2 = figure(plot_width=300, plot_height=250)
p2.line(x=freqs, y=psd, color="black")
p2.xaxis.axis_label = "Freq (Hz)"
p2.yaxis.axis_label = "Power (AU)"
show(row(p1, p2))

INFO       No numerical integration method specified for group 'neurongroup', using method 'euler' (took 0.09s, trying other methods took 0.00s). [brian2.stateupdaters.base.method_choice]


Starting simulation at t=0. s for a duration of 2.5 s
2.5 s (100%) simulated in < 1s


The farther the `tau_e` is from `tau_i` the less stable the oscillation is.

In [88]:
# Noise, with increased tau_e and decreased tau_i
X = model.wc(t, 5, P=5, tau_e=60e-3, tau_i=20e-3, sigma=.01)
x = X[0, :]

# -
freqs, psd = welch(x, int(1 / 1e-3), scaling='density', nperseg=1000/2)
m = np.logical_and(freqs > 2, freqs < 100)
freqs = freqs[m]
psd = psd[m]

# -
p1 = figure(plot_width=600, plot_height=300)
p1.line(x=times, y=x, color="black")
p1.xaxis.axis_label = "Time (s)"
p1.yaxis.axis_label = "Activity"

p2 = figure(plot_width=300, plot_height=300)
p2.line(x=freqs, y=psd, color="black")
p2.xaxis.axis_label = "Freq (Hz)"
p2.yaxis.axis_label = "Power (AU)"
show(row(p1, p2))

INFO       No numerical integration method specified for group 'neurongroup_1', using method 'euler' (took 0.11s, trying other methods took 0.00s). [brian2.stateupdaters.base.method_choice]


Starting simulation at t=0. s for a duration of 2.5 s
2.5 s (100%) simulated in < 1s


### Connectivity

Synapyic time constants are useful in tuning oscillation frequency and stability. To change waveform shape, and oscillatory power, the connections strengths `{c1, c2, c3, c4}` are quite useful.

Let's focus on beta.

In [89]:
# Set a shorter integration time to see what's going on.
t = 0.5
times = model.create_times(t, 1e-3)

In [90]:
X = model.wc(t, 1, P=4,  c1=15.0, c2=15.0, c3=15.0, c4=3.0, tau_e=5e-3, tau_i=20e-3, sigma=.01)
x1 = X[0, :]

# -
freqs, psd = welch(x, int(1 / 1e-3), scaling='density', nperseg=1000/2)
m = np.logical_and(freqs > 2, freqs < 100)
freqs = freqs[m]
psd = psd[m]

# -
p1 = figure(plot_width=500, plot_height=250)
p1.line(x=times, y=x, color="black")
p1.xaxis.axis_label = "Time (s)"
p1.yaxis.axis_label = "Activity"

p2 = figure(plot_width=300, plot_height=250)
p2.line(x=freqs, y=psd, color="black")
p2.xaxis.axis_label = "Freq (Hz)"
p2.yaxis.axis_label = "Power (AU)"
show(row(p1, p2))

INFO       No numerical integration method specified for group 'neurongroup', using method 'euler' (took 0.13s, trying other methods took 0.00s). [brian2.stateupdaters.base.method_choice]


Starting simulation at t=0. s for a duration of 0.5 s
0.5 s (100%) simulated in < 1s


WARNING    /Users/ejp/anaconda/lib/python3.6/site-packages/bokeh/models/sources.py:89: BokehUserWarning: ColumnDataSource's columns must be of the same length
  lambda: warnings.warn("ColumnDataSource's columns must be of the same length", BokehUserWarning))
 [py.warnings]


### Increase power with c1

- c1 is the connection from E to E. It can act akin to a gain control.

In [91]:
X = model.wc(t, 1, P=4, c1=18.0, c2=15.0, c3=15.0, c4=3.0, tau_e=5e-3, tau_i=20e-3, sigma=.01)
x2 = X[0, :]

# -
p1 = figure(plot_width=500, plot_height=250)
p1.line(x=times, y=x1, color="black", legend="Baseline")
p1.line(x=times, y=x2, color="red", legend="c1")
p1.xaxis.axis_label = "Time (s)"
p1.yaxis.axis_label = "Activity"
show(p1)

INFO       No numerical integration method specified for group 'neurongroup_1', using method 'euler' (took 0.08s, trying other methods took 0.00s). [brian2.stateupdaters.base.method_choice]


Starting simulation at t=0. s for a duration of 0.5 s
0.5 s (100%) simulated in < 1s


In [92]:
X = model.wc(t, 1, P=4, c1=10, c2=15.0, c3=15.0, c4=3.0, tau_e=5e-3, tau_i=20e-3, sigma=.01)
x2 = X[0, :]

# -
p1 = figure(plot_width=500, plot_height=250)
p1.line(x=times, y=x1, color="black", legend="Baseline")
p1.line(x=times, y=x2, color="red", legend="c1")
p1.xaxis.axis_label = "Time (s)"
p1.yaxis.axis_label = "Activity"
show(p1)

INFO       No numerical integration method specified for group 'neurongroup', using method 'euler' (took 0.10s, trying other methods took 0.00s). [brian2.stateupdaters.base.method_choice]


Starting simulation at t=0. s for a duration of 0.5 s
0.5 s (100%) simulated in < 1s


- Notice how increasing/decreasing power also alters frequency, and therefore the phase relationship. This illustrates a general property of interdeedence. In a `sinwave` model oscillatory properties are independent. You can change power and leave frequency, or phase intact. This is rarely true in even slightly biological models of oscillation. You can achieve indpendence, but it often requires careful independnce.

- Going too small with `c1` crosses a critical threshold, and the oscillation is lost. Only fixed point behavoir is observed.

In [93]:
X = model.wc(t, 1, P=4, c1=3, c2=15.0, c3=15.0, c4=3.0, tau_e=5e-3, tau_i=20e-3, sigma=.0)
x2 = X[0, :]

# -
p1 = figure(plot_width=500, plot_height=250)
p1.line(x=times, y=x1, color="black", legend="Baseline")
p1.line(x=times, y=x2, color="red", legend="c1")
p1.xaxis.axis_label = "Time (s)"
p1.yaxis.axis_label = "Activity"
show(p1)

INFO       No numerical integration method specified for group 'neurongroup_1', using method 'euler' (took 0.10s, trying other methods took 0.00s). [brian2.stateupdaters.base.method_choice]


Starting simulation at t=0. s for a duration of 0.5 s
0.5 s (100%) simulated in < 1s


### Changing waveform shape with {c2, c4}

You can alter several aspects of the oscillations shape by altering the connectivity between populations.

In [94]:
X = model.wc(t, 1, P=4, c1=15.0, c2=40.0, c3=15.0, c4=5.0, tau_e=5e-3, tau_i=20e-3, sigma=.01)
x3 = X[0, :]

# -
p1 = figure(plot_width=500, plot_height=250)
p1.line(x=times, y=x1, color="black", legend="Baseline")
p1.line(x=times, y=x3, color="red", legend="c2")
p1.xaxis.axis_label = "Time (s)"
p1.yaxis.axis_label = "Activity"
show(p1)

INFO       No numerical integration method specified for group 'neurongroup', using method 'euler' (took 0.11s, trying other methods took 0.00s). [brian2.stateupdaters.base.method_choice]


Starting simulation at t=0. s for a duration of 0.5 s
0.5 s (100%) simulated in < 1s


- Notice the much sharper rise, slower decline, as well as the longer duty cycle. (And that the frequency increased, while power declined)

## Bias and shape

Increasing the static bias term in the model (`P`) can also alter waveform shape (and the type of oscillator, in a dynamical systems systems sense - limit cycle to a homoclinic orbit).

In [95]:
X = model.wc(t, 1, P=7, c1=15.0, c2=15.0, c3=15.0, c4=3.0, tau_e=5e-3, tau_i=20e-3, sigma=.01)
x4 = X[0, :]

# -
p1 = figure(plot_width=500, plot_height=250)
p1.line(x=times, y=x1, color="black", legend="Baseline")
p1.line(x=times, y=x4, color="red", legend="P")
p1.xaxis.axis_label = "Time (s)"
p1.yaxis.axis_label = "Activity"
show(p1)

INFO       No numerical integration method specified for group 'neurongroup_1', using method 'euler' (took 0.11s, trying other methods took 0.00s). [brian2.stateupdaters.base.method_choice]


Starting simulation at t=0. s for a duration of 0.5 s
0.5 s (100%) simulated in < 1s


- But again, `P` is has critical values. Above and below these we get fixed behavoir.

In [96]:
X = model.wc(t, 1, P=.1, c1=15.0, c2=15.0, c3=15.0, c4=3.0, tau_e=5e-3, tau_i=20e-3, sigma=.0)
x4 = X[0, :]

# -
p1 = figure(plot_width=500, plot_height=250)
p1.line(x=times, y=x1, color="black", legend="Baseline")
p1.line(x=times, y=x4, color="red", legend="P")
p1.xaxis.axis_label = "Time (s)"
p1.yaxis.axis_label = "Activity"
show(p1)

INFO       No numerical integration method specified for group 'neurongroup', using method 'euler' (took 0.10s, trying other methods took 0.00s). [brian2.stateupdaters.base.method_choice]


Starting simulation at t=0. s for a duration of 0.5 s
0.5 s (100%) simulated in < 1s


In [97]:
X = model.wc(t, 1, P=8, c1=15.0, c2=15.0, c3=15.0, c4=3.0, tau_e=5e-3, tau_i=20e-3, sigma=.0)
x4 = X[0, :]

# -
p1 = figure(plot_width=500, plot_height=250)
p1.line(x=times, y=x1, color="black", legend="Baseline")
p1.line(x=times, y=x4, color="red", legend="P")
p1.xaxis.axis_label = "Time (s)"
p1.yaxis.axis_label = "Activity"
show(p1)

INFO       No numerical integration method specified for group 'neurongroup_1', using method 'euler' (took 0.10s, trying other methods took 0.00s). [brian2.stateupdaters.base.method_choice]


Starting simulation at t=0. s for a duration of 0.5 s
0.5 s (100%) simulated in < 1s


## The Kuramoto model

The [kuramoto model](https://www.ncbi.nlm.nih.gov/pubmed/21151358) is defined on the unit circle. It has been used for 50 years to study systems of coupled oscillators. It's a great tool to study phase sync between electrodes. 

In it's most simlified for, the Kuramoto equation is

    theta_n' = omega_n + (K / N) \sum_{m=1}^{N} sin(theta_m - theta_n) + N(0, sigma)
  
This version has N paramters:

1. N : # of oscillators
2. omega : the average frequency oscillators in the system
3. K : the coupling strength
4. sigma: standard deviation of the Gaussian noise term

Where

- `N(0, 1)` is a scalar sample taken from a Gaussian
- All coupling strengths are equal (i.e. K/ M rather than K_m / N)
- Coupling is all to all
- All omegas are the same for each oscillatory

(All three of these latter conditions can be relaxed, if it of interest.)

In [98]:
# Reset to 2.0 seconds
t = 2
times = model.create_times(t, 1e-3)

With no noise (`sigma`) and strong coupling (`K`) the model quickly phase-synchornizes.

In [99]:
K = 6
sigma = 0.01
X = model.kuramoto(t, 10, 10, K, sigma=sigma)

# -
p1 = figure(plot_width=500, plot_height=250)
for i in range(10):
    x = X[i, :]
    p1.line(x=times, y=x, color="black", alpha=0.3)
    p1.xaxis.axis_label = "Time (s)"
    p1.yaxis.axis_label = "Activity"

p2 = figure(plot_width=300, plot_height=250)
for i in range(10):
    x = X[i, :]
    freqs, psd = welch(x, int(1 / 1e-3), scaling='density', nperseg=1000/2)
    m = np.logical_and(freqs > 2, freqs < 100)
    freqs = freqs[m]
    psd = psd[m]

    p2.line(x=freqs, y=psd, color="black")
    p2.xaxis.axis_label = "Freq (Hz)"
    p2.yaxis.axis_label = "Power (AU)"
show(row(p1, p2))

Adding (a lot) of noise will desynchronize the model.

In [100]:
K = 6
sigma = 2.5
X = model.kuramoto(t, 10, 10, K, sigma=sigma)

# -
p1 = figure(plot_width=500, plot_height=250)
for i in range(10):
    x = X[i, :]
    p1.line(x=times, y=x, color="black", alpha=0.3)
    p1.xaxis.axis_label = "Time (s)"
    p1.yaxis.axis_label = "Activity"

p2 = figure(plot_width=300, plot_height=250)
for i in range(10):
    x = X[i, :]
    freqs, psd = welch(x, int(1 / 1e-3), scaling='density', nperseg=1000/2)
    m = np.logical_and(freqs > 2, freqs < 100)
    freqs = freqs[m]
    psd = psd[m]

    p2.line(x=freqs, y=psd, color="black")
    p2.xaxis.axis_label = "Freq (Hz)"
    p2.yaxis.axis_label = "Power (AU)"
show(row(p1, p2))

As will taking `K` sub-critical.

In [101]:
K = 1
sigma = 0.01
X = model.kuramoto(t, 10, 10, K, sigma=sigma)

# -
p1 = figure(plot_width=500, plot_height=250)
for i in range(10):
    x = X[i, :]
    p1.line(x=times, y=x, color="black", alpha=0.3)
    p1.xaxis.axis_label = "Time (s)"
    p1.yaxis.axis_label = "Activity"

p2 = figure(plot_width=300, plot_height=250)
for i in range(10):
    x = X[i, :]
    freqs, psd = welch(x, int(1 / 1e-3), scaling='density', nperseg=1000/2)
    m = np.logical_and(freqs > 2, freqs < 100)
    freqs = freqs[m]
    psd = psd[m]

    p2.line(x=freqs, y=psd, color="black")
    p2.xaxis.axis_label = "Freq (Hz)"
    p2.yaxis.axis_label = "Power (AU)"
show(row(p1, p2))

Weakening coupling and adding noise is quite catastrophic

In [102]:
K = 3
sigma = 2
X = model.kuramoto(t, 10, 10, K, sigma=sigma)

# -
p1 = figure(plot_width=500, plot_height=250)
for i in range(10):
    x = X[i, :]
    p1.line(x=times, y=x, color="black", alpha=0.3)
    p1.xaxis.axis_label = "Time (s)"
    p1.yaxis.axis_label = "Activity"

p2 = figure(plot_width=300, plot_height=250)
for i in range(10):
    x = X[i, :]
    freqs, psd = welch(x, int(1 / 1e-3), scaling='density', nperseg=1000/2)
    m = np.logical_and(freqs > 2, freqs < 100)
    freqs = freqs[m]
    psd = psd[m]

    p2.line(x=freqs, y=psd, color="black")
    p2.xaxis.axis_label = "Freq (Hz)"
    p2.yaxis.axis_label = "Power (AU)"
show(row(p1, p2))

## PAC

TODO - Expand the EI model to general PAC. For [example](http://link.springer.com/10.1007/s00422-016-0687-5)

 ## LIF
    
TODO - 

Create a two-population model of randomly, sparse, connected LIF neurons. Subject this population to osoillatory modulation, recording an estimage of the LFP.

## Basal Ganglia

TODO - Gurney produced a mass model of the major BG nuclei. (I've never implemted this, so it may take a bit')